In [1]:
from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import r2_score
import seaborn as sns


In [2]:
data = pd.read_csv('winemag-data-130k-v2.csv')
#data.head()
df = data.copy()
df = df[['Unnamed: 0','country', 'description']]
df.set_index('Unnamed: 0').head()
df.dropna(inplace = True)

In [3]:
dff = df.copy()
dff.country[dff.country == 'Italy'] = 0
dff.country[dff.country == 'Portugal'] = 1
dff.country[dff.country == 'US'] = 2
dff.country[dff.country == 'Spain'] = 3
dff.country[dff.country == 'France'] = 4
dff.country[dff.country == 'Germany'] = 5
dff.country[dff.country == 'Argentina'] = 6
dff.country[dff.country == 'Chile'] = 7
dff.country[dff.country == 'Australia'] = 8
dff.country[dff.country == 'Austria'] = 9
dff.country[dff.country == 'South Africa'] = 10
dff.country[dff.country == 'New Zealand'] = 11
dff.country[dff.country == 'Israel'] = 12
dff.country[dff.country == 'Hungary'] = 13
dff.country[dff.country == 'Greece'] = 14
dff.country[dff.country == 'Romania'] = 15
dff.country[dff.country == 'Mexico'] = 16
dff.country[dff.country == 'Canada'] = 17
dff.country[dff.country == 'Turkey'] = 18
dff.country[dff.country == 'Czech Republic'] = 19
dff.country[dff.country == 'Luxembourg'] = 20
dff.country[dff.country == 'Georgia'] = 21
dff.country[dff.country == 'Uruguay'] = 22
dff.country[dff.country == 'England'] = 23
dff.country[dff.country == 'Lebanon'] = 24
dff.country[dff.country == 'Serbia'] = 25
dff.country[dff.country == 'Brazil'] = 26
dff.country[dff.country == 'Moldova'] = 27
dff.country[dff.country == 'Morocco'] = 28
dff.country[dff.country == 'India'] = 29
dff.country[dff.country == 'Bulgaria'] = 30
dff.country[dff.country == 'Cyprus'] = 31
dff.country[dff.country == 'Armenia'] = 32
dff.country[dff.country == 'Slovenia'] = 33
dff.country[dff.country == 'Croatia'] = 34
dff.country[dff.country == 'Peru'] = 35
dff.country[dff.country == 'Switzerland'] = 36
dff.country[dff.country == 'Bosnia and Herzegovina'] = 37
dff.country[dff.country == 'Ukraine'] = 38
dff.country[dff.country == 'Slovakia'] = 39
dff.country[dff.country == 'Macedonia'] = 40
dff.country[dff.country == 'China'] = 41
dff.country[dff.country == 'Egypt'] = 42
dff.country.unique()

C:\Users\FinTech\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\FinTech\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\FinTech\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\FinTech\Anacon

C:\Users\FinTech\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\FinTech\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\FinTech\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\FinTech\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 33, 20, 34, 21, 22, 23, 24, 25, 26, 27, 28, 35, 29, 30, 31, 32,
       36, 37, 38, 39, 40, 41, 42], dtype=object)

In [4]:
dff = dff[0: 5000]
dff.head().set_index('Unnamed: 0')


,country,description
Unnamed: 0,,
0,0,"Aromas include tropical fruit, broom, brimston..."
1,1,"This is ripe and fruity, a wine that is smooth..."
2,2,"Tart and snappy, the flavors of lime flesh and..."
3,2,"Pineapple rind, lemon pith and orange blossom ..."
4,2,"Much like the regular bottling from 2012, this..."


In [5]:
y = dff.country.astype(int)

In [6]:
corpus = dff['description']
vectorizer = CountVectorizer(ngram_range=(1, 1), min_df = 1, binary = True)
X = vectorizer.fit_transform(corpus)
X = pd.DataFrame(data = X.toarray(), columns = vectorizer.get_feature_names())
x_names = X.sum(axis = 0).sort_values(ascending = False).head(2500).reset_index(inplace=False)
word_list = list(x_names['index'])
Xs = X[word_list]
Xs.head()

,and,the,of,this,with,is,it,wine,flavors,in,...,bee,ice,miss,pinotage,sprinkling,veers,clover,via,aren,lane
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.30, random_state=42)
rforest = RandomForestClassifier(n_estimators = 1000, n_jobs=-1)
rforest_model = rforest.fit(X_train,y_train)
preds_class_rf = rforest_model.predict(X_test)
#print(confusion_matrix(y_test,preds_class_rf))
print(rforest_model.score(X_test, y_test))

0.678


In [118]:
#Instead of having 43 unique countries, we rather make two categories of countries when the First alphabet lies
#between A-K, then we represent it as 0 and then if it lies between L-Z then we make it 1

In [107]:
name_alp = df.country.str.lower()
lst=[]
for i in name_alp:
    alp = i[0]
    #print(alp)
    class1 = ['a','b', 'c','d','e','f','g','j','h','i','j','k']
    class2 = ['lmnopqrstuvwxyz']
    if i[0] in class1:
        zero = 0
        lst.append(0)
        print(zero)
    else:
        one = 1
        lst.append(1)
        print(one)
#     lst.append(zero, one)

0
1
1
1
1
1
0
0
0
0
1
0
1
0
1
0
0
0
1
1
1
1
0
1
0
1
0
0
0
1
0
0
0
1
1
1
0
0
0
0
0
1
0
1
0
1
0
1
1
0
0
0
0
0
0
1
1
0
0
1
1
0
1
0
1
0
0
1
1
0
1
1
0
1
1
1
0
0
1
1
0
1
0
0
1
0
1
1
0
0
1
1
1
0
1
0
0
1
0
1
1
1
1
0
0
0
0
0
1
0
0
1
0
0
1
1
1
1
0
0
0
1
1
0
1
1
0
0
0
1
0
0
1
0
1
0
0
1
0
0
0
0
1
0
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
0
0
1
1
0
0
0
0
1
1
1
0
0
0
1
1
0
0
0
1
0
1
1
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
1
1
1
1
1
1
0
1
0
1
1
1
0
0
1
1
0
1
0
0
0
0
1
1
1
1
1
1
0
0
1
0
1
1
0
0
1
0
1
1
1
1
0
0
1
1
1
1
1
1
0
1
1
1
1
1
0
1
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
1
0
1
0
0
1
1
0
1
0
0
0
0
0
1
0
1
0
0
1
1
0
0
0
1
1
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
1
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
1
1
1
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
0
1
1
1
0
0
0
1
0
0
0
0
0
1
1
0
1
1
1
1
0
1
1
1
1
1
0
0
0
1
1
1
1
0
1
1
0
0
1
0
0
0
0
0
0
0
0
1
1
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
1
0
0
0
1
0
1
1
1
0
1
0
0
0
1
0
1
1
1
1
1
1
1
0


0
0
1
0
1
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
0
1
1
1
0
1
1
1
1
1
0
1
1
1
0
0
0
0
1
1
1
0
0
0
0
0
0
1
1
1
1
0
1
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
0
1
0
0
1
0
0
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
0
0
0
1
1
1
1
1
0
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
1
0
1
1
0
1
1
0
0
0
0
1
1
1
0
1
1
0
0
0
1
1
1
0
1
0
0
1
0
1
1
0
0
1
1
0
1
1
0
1
0
0
1
0
0
1
1
0
1
1
1
1
1
1
0
0
1
0
1
0
0
0
0
0
0
1
1
1
0
1
1
0
0
1
1
0
1
1
0
0
0
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
1
1
1
0
1
1
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
0
1
1
1
0
0
1
0
0
1
0
1
1
0
1
1
0
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
0
1
1
1
0
1
0
0
1
1
0
0
0
1
1
0
0
0
0
0
1
0
1
0
1
0
1
0
1
1
1
0
1
0
0
0
1
0
1
0
0
0
1
1
0
1
0
0
0
1
1
1
1
1
1
0
1
1
1
0
0
0
1
0
0
1
0
1
0
0
0
0
1
1
0
0
0
1
0
0
0
1
0
1
0
0
1
0
1
0
1
1
1
1
1
0
0
1
1
1
0
0
1
0
1
1
0
1
0
1
0
0
0
1
0
0
1
0
0
0
1
1
1
0
1
1
1
0
1
0
1
0
1
1
1
0
1
1
0
1
0
0
0
1
0
0
1
1
0
1
1
0
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
0
1
0
0
0
0
0
0
0


1
0
1
0
1
1
1
0
0
0
1
1
1
1
1
1
0
0
0
1
1
0
1
1
0
0
1
1
1
0
0
0
0
1
0
1
0
0
0
1
0
0
1
0
0
1
1
1
0
1
0
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1
1
1
0
1
0
1
0
0
1
0
1
0
0
0
1
1
1
1
0
0
0
0
1
0
0
1
0
0
0
1
1
0
0
1
1
1
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
1
0
0
1
0
0
0
0
1
0
0
1
0
0
1
1
0
1
0
0
0
0
0
1
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
0
0
1
0
1
1
0
1
1
1
0
1
0
0
1
0
1
0
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
0
1
0
0
0
0
0
1
1
0
0
1
1
1
1
0
0
1
0
1
1
1
0
0
0
1
0
1
0
0
0
1
0
1
1
0
1
0
1
0
0
1
0
0
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
0
0
0
1
0
1
1
1
1
1
1
1
1
0
0
1
0
1
0
0
1
1
0
0
1
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
0
0
0
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
0
0
0
0
1
1
1
0
1
1
0
0
1
0
1
1
1
1
0
1
0
1
0
1
1
1
0
1
1
0
0
0
1
0
0
0
0
1
0
0
0
1
1
1
1
1
0
0
0
0
0
1
0
1
0
1
1
1
1
0
0
0
1
0
0
0
1
1
1
0
0
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
0
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
0
0
0
1
0
1
1
1
1
1
1
1
1
1
1
0


1
0
1
1
0
1
1
1
1
0
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
0
0
0
0
1
1
0
1
0
0
0
1
0
1
1
1
1
1
1
1
1
0
0
1
1
1
0
0
1
1
1
1
1
0
1
1
1
1
1
0
1
0
1
1
0
1
1
1
1
1
0
0
0
0
1
1
0
0
0
0
0
1
1
1
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
1
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
1
0
1
1
1
0
1
0
0
0
1
1
0
1
0
0
0
1
1
0
0
0
0
1
0
0
1
1
0
1
1
1
0
1
0
0
1
1
1
1
0
0
1
1
1
0
0
0
1
1
0
1
0
0
0
0
1
1
1
0
0
0
1
1
0
0
0
0
1
1
0
0
0
1
0
0
0
0
1
0
0
1
1
1
0
1
1
1
0
0
1
0
1
1
1
1
1
1
0
0
0
1
0
0
0
1
1
0
1
1
1
1
1
1
0
1
1
0
0
0
0
0
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
0
0
1
1
1
1
1
0
0
1
0
1
0
0
0
1
0
1
1
0
1
1
0
0
0
1
1
1
1
0
0
0
0
0
1
1
1
0
0
0
0
0
1
0
0
0
1
0
1
0
1
1
1
0
1
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
1
1
0
0
0
0
1
0
1
1
0
1
0
1
1
1
1
1
0
1
1
0
0
0
1
1
1
0
0
0
0
1
1
1
0
0
0
0
1
1
1
1
0
1
0
0
1
0
1
1
0
1
1
1
0
1
0
0
0
1
1
0
1
1
0
1
1
1
1
1
1
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
1
0
0
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
0
1
0


1
0
1
0
1
0
1
0
0
1
0
1
0
1
1
1
1
0
0
0
1
1
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
1
0
0
1
0
0
1
1
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
1
1
0
1
1
1
1
1
1
1
1
0
0
0
0
0
1
1
0
1
0
0
0
1
1
1
0
0
0
0
0
0
1
0
1
1
1
1
1
1
1
1
0
0
1
0
1
1
1
1
1
0
1
1
0
1
0
0
0
0
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
0
1
0
0
1
1
0
0
0
0
0
1
0
1
0
1
0
1
1
0
0
1
1
1
0
0
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
0
1
0
1
1
0
1
0
1
0
1
1
1
0
1
1
1
0
1
1
1
0
0
0
0
0
0
1
0
0
1
0
1
1
1
0
0
0
1
1
0
1
1
1
0
0
0
0
1
1
0
1
0
1
0
1
1
1
1
0
0
1
0
0
1
1
0
1
1
1
0
0
0
0
1
1
1
1
1
0
1
1
0
1
1
0
0
0
1
1
0
0
0
1
1
0
0
1
1
1
0
1
0
1
0
0
1
1
1
1
0
0
1
0
0
0
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
0
0
1
1
1
0
0
0
1
1
0
0
1
0
0
0
0
0
1
1
0
1
0
1
1
0
1
0
1
0
1
0
0
1
0
1
1
1
1
1
0
1
0
1
1
0
0
1
1
0
1
1
1
0
1
1
0
0
0
1
0
0
0
0
1
0
1
1
1
1
1
0
0
0
0
1
1
1
1
1
0
1
1
0
0
1
1
0
0
0
0
1
1
1
1
1
1
1
0
0
1
1
0
1
1
1
1
1
0
0
1
0
0
1
1
0
0
1
1
0
0
1
0
1
1
0
0
1
1
1
0
1
0
1
1
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1


...

0
0
1
0
1
1
1
0
0
1
1
1
1
1
0
1
0
0
1
1
0
0
1
0
1
1
1
1
1
0
0
1
0
0
0
0
0
1
1
1
1
1
0
1
0
0
0
0
0
0
1
1
1
0
1
0
0
1
1
1
1
0
0
1
0
1
0
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
0
0
1
0
0
0
1
1
1
1
1
1
1
0
1
0
1
1
1
1
0
0
1
0
0
0
1
0
0
1
0
1
0
1
1
1
0
1
1
0
0
0
1
1
0
0
1
0
1
0
0
1
1
1
1
0
1
1
1
1
0
1
1
0
1
1
0
0
1
1
0
1
1
0
0
1
1
1
1
1
1
0
0
1
1
0
0
0
0
0
1
0
0
1
1
1
0
0
0
0
1
0
1
1
0
1
1
0
1
1
1
0
1
0
1
1
1
0
0
1
0
1
1
1
0
1
0
1
1
1
1
0
1
0
1
1
1
1
1
0
0
1
1
1
0
0
1
1
1
0
1
1
1
1
0
0
0
0
1
0
0
0
0
0
1
0
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
0
1
0
0
1
0
0
0
0
0
1
1
0
1
0
1
1
1
0
0
0
1
1
0
0
1
0
1
0
0
1
1
0
1
0
0
1
1
1
0
1
1
1
1
1
0
1
0
1
1
0
0
1
0
1
1
0
0
0
1
1
0
0
0
1
0
0
0
0
0
0
0
1
1
1
0
1
0
0
1
1
1
0
1
1
1
1
0
1
0
0
1
0
0
1
1
1
1
1
0
0
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
0
1
1
1
0
0
1
0
1
0
0
1
1
1
1
0
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
0
0
0
1
1
0
1
1
1
0
1
0
0
1
1
1
1
0
1
0
0
0
0
0
1
1
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
1
1
1
0


0
0
0
0
1
0
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
1
1
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
1
0
1
1
0
1
1
1
1
1
1
0
1
0
1
1
1
0
0
1
1
1
1
1
1
0
1
1
0
1
0
1
0
1
0
0
0
1
1
1
1
1
0
1
0
0
1
0
0
1
0
0
0
0
1
0
0
1
0
0
1
1
1
0
1
1
0
0
0
0
1
0
0
1
1
1
1
1
0
0
0
1
0
1
0
1
1
0
1
1
1
0
0
1
0
1
1
1
0
0
1
1
1
1
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
1
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
1
0
1
0
1
1
0
1
1
0
0
1
1
0
1
0
1
1
0
0
1
1
0
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
0
1
1
0
0
0
0
0
0
1
1
1
1
0
1
1
1
1
0
1
0
0
0
0
0
0
0
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
0
0
0
1
1
0
0
1
1
0
0
0
1
0
0
1
1
1
0
1
0
0
1
0
1
1
1
1
1
0
0
0
0
1
0
0
0
1
1
1
0
1
0
0
0
1
1
1
1
0
0
1
0
0
0
1
0
1
1
1
1
1
1
0
1
1
0
0
0
0
1
1
1
0
0
1
0
0
0
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
0
0
0
1
1
1
1
0
0
1
1
1
1
1
1
1
0
0
0
1
0
1
1
1
0
0
1
0
1
0
1
1
1
1
0
1
0
1
0
0
0
1
0
1
0
0
1
1
1
0
0
0
0
1
1
1
1


0
0
1
0
0
0
0
0
1
1
1
0
0
1
1
1
1
0
0
0
1
0
1
1
1
1
1
1
1
0
0
1
1
0
0
0
1
1
1
0
1
1
1
0
0
1
0
1
0
1
1
0
1
1
1
1
0
1
0
1
0
0
0
0
1
1
0
0
1
0
0
0
0
0
0
0
0
1
0
0
1
0
1
1
1
1
1
1
1
0
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
0
1
1
0
1
0
1
1
1
0
0
0
0
0
0
0
1
1
1
0
1
0
0
1
1
1
1
0
0
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
0
1
0
1
1
1
1
0
1
0
1
0
0
1
1
1
1
1
0
1
0
1
0
0
0
0
1
1
1
1
0
0
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
0
0
1
1
0
1
1
0
0
0
0
1
1
1
0
1
1
0
0
0
0
1
1
0
1
0
1
1
0
1
1
1
1
0
0
1
1
0
0
0
0
1
1
1
0
0
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
0
1
1
1
1
0
1
1
1
0
0
1
1
1
1
0
0
0
1
0
0
1
1
0
0
0
0
1
0
0
1
0
1
0
0
1
1
1
0
0
1
0
0
1
1
0
1
1
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
0
0
0
0
0
1
0
0
0
1
1
0
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
0
0
1
0
0
0
0
0
1
1
1
1
0
0
0
0
1
0
0
0
1
1
0
1
0
1
0
1
1
0
1
1
1
0
0
0
1
0
1
0
1
1
1
1
1
1
1
1
0
0
1
0
0
0
0
0
1
1
1
1
0
0
1
1
0
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
0
0
1
0
0
1
0
0
1
1
1


0
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
1
1
1
0
0
0
0
0
0
1
0
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
0
1
0
1
0
0
0
1
1
0
0
0
0
1
1
0
1
0
1
1
1
1
0
1
1
0
0
0
0
1
0
1
1
0
1
1
0
0
0
1
0
1
1
1
1
0
1
1
0
0
0
0
1
0
0
1
0
1
1
0
0
0
1
0
1
1
1
1
0
0
1
1
1
0
0
0
1
1
1
1
0
1
1
1
0
0
0
1
1
1
0
1
1
1
1
1
1
1
0
0
0
1
1
1
1
0
1
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
1
0
0
1
0
0
0
0
1
0
1
1
0
0
1
0
0
1
1
1
1
0
0
0
1
0
0
1
1
0
1
0
1
1
1
0
0
0
0
0
1
0
0
0
0
1
1
1
1
1
0
1
1
1
0
1
1
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
1
0
0
1
1
1
1
0
0
0
1
1
1
1
0
1
1
0
1
1
0
1
1
1
1
0
0
0
1
0
1
1
0
1
0
0
0
0
0
1
1
1
0
0
1
1
0
1
0
1
0
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
0
0
0
0
0
1
1
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
1
1
1
1
1
0
1
0
1
1
1
0
0
1
0
0
0
1
1
1
1
1
1
1
0
1
1
1
0
1
1
0
1
0
1
0
1
0
0
1
1
0
1
0
1
1
0
1
1
1
0
1
0
0
0
1
0
1
0
1
0
1
1
0
0
0
0
1
0
1
1
1
0
1
0
1
1
0
0
0
1
1
1
0
0
1
1
1
0
1
1
0
1
0
1
1
1
0
1
0
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1


0
0
1
1
1
0
0
1
0
0
0
1
1
1
0
1
0
0
0
1
0
0
0
1
1
0
0
0
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
0
0
1
0
1
0
1
1
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
1
0
1
1
1
0
1
0
0
0
1
1
1
0
1
0
1
1
1
1
0
0
0
1
1
1
0
0
1
1
0
0
0
1
1
0
0
0
0
1
1
1
1
0
0
1
1
1
0
0
1
1
0
1
1
1
1
1
1
0
0
1
0
1
0
1
1
1
1
1
0
0
0
0
0
1
0
1
0
0
1
0
0
1
0
1
1
1
1
1
0
0
0
1
1
0
1
1
1
0
1
1
0
1
0
0
0
1
0
0
1
1
0
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
0
0
0
1
1
0
1
0
1
0
1
0
0
1
1
1
0
1
1
0
0
0
1
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
1
1
0
1
0
0
0
0
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
1
0
0
0
0
1
1
0
1
0
1
1
1
0
0
0
1
0
0
0
1
0
1
1
1
1
0
1
0
0
0
1
0
0
1
0
1
0
0
1
1
1
1
1
0
0
0
1
0
0
1
0
1
1
0
1
1
1
1
1
0
0
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
1
0
0
0
1
0
1
0
1
0
1
1
0
0
1
1
1
0
1
0
0
1
0
1
0
1
0
0
0
0
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
0
1
0
1
1
0
0
1
1
0
0
0
0
1
0
1
1
0
1
0
1
0
0
0
0
1
1
1
0
1
1
1
1


In [117]:
#lst

In [116]:
#we then took first 5000 of from the list and with already made features as Xs, we predicted new y as countries [0,1]
new_lst = lst[0:5000]
len(new_lst)
#len(Xs)
X_train, X_test, y_train, y_test = train_test_split(Xs, new_lst, test_size=0.30, random_state=42)
rforest = RandomForestClassifier(n_estimators = 1000, n_jobs=-1)
rforest_model = rforest.fit(X_train,y_train)
preds_class_rf = rforest_model.predict(X_test)
#print(confusion_matrix(y_test,preds_class_rf))
print(rforest_model.score(X_test, y_test))

0.7906666666666666
